# Searching GA4GH VariantAnnotations using user-provided SO maps

This code demonstrates how to search ga4gh schema using a named maps of SO ids. Possible uses include classififcation of variation by "impact", as SnpEff does, or classification variation by region.

The results here may be compared with the survey of terms from the same region in [Searching by SO terms](Searching%20by%20SO%20terms.ipynb)

In [34]:
import itertools
import pandas as pd
from pivottablejs import pivot_ui

import ga4gh.client
print(ga4gh.__version__)

gc = ga4gh.client.HttpClient("http://localhost:8000")

0.1.dev632+ncb43455c1003


In [35]:
region_constraints = dict(referenceName="1", start=0, end=int(1e10))
variant_set_id = 'YnJjYTE6T1I0Rg'
variant_annotation_sets = list(gc.searchVariantAnnotationSets(variant_set_id))
variant_annotation_set = variant_annotation_sets[0]
print("Using first variant annotation set (of {n} total) for variant set {vs_id}\nvas_id={vas.id}".format(
    n=len(variant_annotation_sets), vs_id=variant_set_id, vas=variant_annotation_set))

Using first variant annotation set (of 1 total) for variant set YnJjYTE6T1I0Rg
vas_id=YnJjYTE6T1I0Rjp2YXJpYW50YW5ub3RhdGlvbnM


In [36]:
def _mk_effect_id_filter(so_ids=[]):
    """return list of OntologyTerm effect filters for the given list of so ids

    >>> print(_mk_effect_id_filter("SO:123 SO:456".split()))
    [{'id': 'SO:123'}, {'id': 'SO:456'}]
    """
    return [{"id": id} for id in so_ids]

In [42]:
# poor-man's SO id-to-id map
import requests
import re
id_name_re = re.compile("id: (?P<id>SO:\d+)\nname: (?P<name>\S+)")
url = "https://raw.githubusercontent.com/The-Sequence-Ontology/SO-Ontologies/master/so-xp-simple.obo"
so_name_id_map = {
    m.group('name'): m.group('id')
    for m in (idid_re.search(s) for s in requests.get(url).text.split("\n\n")) if m is not None
    }

## SO Maps
Define two SO maps: "snpeff" and "regional".
"snpeff" classifies SO names by predicted impact
"scope" classifies SO names by scale/scope/region of change 

In [62]:
# Build SnpEff impact map
# derived from
# https://github.com/pcingola/SnpEff/blob/18ad192f751d2e34595949dda8b25c295c8a9ca1/src/main/java/org/snpeff/snpEffect/EffectType.java
# with non-standard SO names removed
# This is why we should force SO ids...

snpeff_so_map = {
    'high': [
        "gene_fusion",
        "stop_gained",
        "stop_lost",
        "start_lost",
    ],

    'moderate': [
        # all SO names were invalid
    ],

    "low": [
        # all SO names were invalid
    ],

    "modifiers": [
        "chromosome",
        "exon",
        "genome",
        "intron",
        "sequence",
        "transcript",
    ]
}

In [63]:
scope_so_map = {
    "locus": [
        "gene_fusion",
        "upstream_gene_variant",
    ],

    "cds": [
        "missense_variant",
        "start_lost",
        "stop_gained",
        "stop_lost",
        "synonymous_variant",
    ],
    
    "utr": [
        "3_prime_UTR_variant",
        "5_prime_UTR_variant", 
    ],
   }

In [64]:
so_maps = {"snpeff": snpeff_so_map, "scope": scope_so_map}

In [70]:
# search for variants so maps and subsets therein 
for map_name in so_maps.keys():
    for so_set_name in so_maps[map_name].keys():
        so_ids = [so_name_id_map[so_name] for so_name in so_maps[map_name][so_set_name]]
        efilter = _mk_effect_id_filter(so_ids)
        vs = [] if len(efilter)==0 else list(gc.searchVariantAnnotations(variant_annotation_set.id, effects=efilter, **region_constraints))
        print("{m:10s} {n:10s} {n_vars:5d} {sos}".format(
                m=map_name, n=so_set_name, n_vars=len(vs), sos=", ".join(so_ids)))


snpeff     high           1 SO:0001565, SO:0001587, SO:0001578, SO:0002012
snpeff     moderate       0 
snpeff     low            0 
snpeff     modifiers      0 SO:0000340, SO:0000147, SO:0001026, SO:0000188, SO:1000082, SO:0000673
scope      utr            0 SO:0001624, SO:0001623
scope      cds           24 SO:0001583, SO:0002012, SO:0001587, SO:0001578, SO:0001819
scope      locus         63 SO:0001565, SO:0001631
